In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from utils.classifications_utils import *
from utils.data_processing_utils_lin_edit import *
from utils.data_visualization_utils import *
from utils.metrics_utils import *
from utils.grid_search_utils import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy import signal
from utils.reduce_noise_utils import *
import pickle
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# Drone 1 Data

In [2]:
#specify Project and drone data
project = "Project 3"
drone_in_use = "drone1"

In [3]:
level_0_wind = load_data(0, 3, project, drone_in_use)
level_0_wind = separate_data_based_on_apparatus(level_0_wind)

level_1_wind = load_data(1, 3, project, drone_in_use)
level_1_wind = separate_data_based_on_apparatus(level_1_wind)

level_2_wind = load_data(2, 3, project, drone_in_use)
level_2_wind = separate_data_based_on_apparatus(level_2_wind)

# level_3_wind = load_data(3, 3, project, drone_in_use)
# level_3_wind = separate_data_based_on_apparatus(level_3_wind)

# level_4_wind = load_data(4, 3, project, drone_in_use)
# level_4_wind = separate_data_based_on_apparatus(level_4_wind)

# # display(level_0_wind.keys())

In [4]:
acc_no_wind, gyro_no_wind, _, stab_no_wind = level_0_wind.values()
label_0 = [0 for x in range(acc_no_wind.shape[0])]

acc_level_1_wind, gyro_level_1_wind, _, stab_level_1 = level_1_wind.values()
label_1 = [1 for x in range(acc_level_1_wind.shape[0])]

acc_level_2_wind, gyro_level_2_wind, _, stab_level_2 = level_2_wind.values()
label_2 = [2 for x in range(acc_level_2_wind.shape[0])]

# acc_level_3_wind, gyro_level_3_wind, _, stab_level_3 = level_3_wind.values()
# label_3 = [3 for x in range(acc_level_3_wind.shape[0])]

# acc_level_4_wind, gyro_level_4_wind, _, stab_level_4 = level_4_wind.values()
# label_4 = [4 for x in range(acc_level_4_wind.shape[0])]

# Split test and train Data

In [5]:
X_train_stab_0, X_test_stab_0, y_train_stab_0, y_test_stab_0 = \
train_test_split(stab_no_wind, label_0, test_size=0.1, shuffle=False)

In [6]:
X_train_stab_1, X_test_stab_1, y_train_stab_1, y_test_stab_1 = \
train_test_split(stab_level_1, label_1, test_size=0.1, shuffle=False)

In [7]:
X_train_stab_2, X_test_stab_2, y_train_stab_2, y_test_stab_2 = \
train_test_split(stab_level_2, label_2, test_size=0.1, shuffle=False)

In [8]:
# X_train_stab_3, X_test_stab_3, y_train_stab_3, y_test_stab_3 = \
# train_test_split(stab_level_3, label_3, test_size=0.1, shuffle=False)

In [9]:
# X_train_stab_4, X_test_stab_4, y_train_stab_4, y_test_stab_4 = \
# train_test_split(stab_level_4, label_4, test_size=0.1, shuffle=False)

# Stack data up

In [10]:
X_train_stab = X_train_stab_0.append(X_train_stab_1)
X_train_stab = X_train_stab.append(X_train_stab_2)
# X_train_stab = X_train_stab.append(X_train_stab_3)
# X_train_stab = X_train_stab.append(X_train_stab_4)
y_train_stab = np.hstack((y_train_stab_0, y_train_stab_1, y_train_stab_2))

In [11]:
X_test_stab = X_test_stab_0.append(X_test_stab_1)
X_test_stab = X_test_stab.append(X_test_stab_2)
# X_test_stab = X_test_stab.append(X_test_stab_3)
# X_test_stab = X_test_stab.append(X_test_stab_4)
y_test_stab = np.hstack((y_test_stab_0, y_test_stab_1, y_test_stab_2))

# Transform and generate features

In [12]:
from transformers.featureGenerator import FeatureGenerator

In [13]:
feature_generator_stab2 = FeatureGenerator(1)
feature_generator_stab2.fit(X_train_stab, 3)

FeatureGenerator(sliding_window=1)

In [14]:
X_train_stab = feature_generator_stab2.transform(X_train_stab, 'stab')
y_train_stab = reduce_label_amount(y_train_stab, 3)

5301


In [15]:
feature_generator_stab_test2 = FeatureGenerator(1)
feature_generator_stab_test2.fit(X_test_stab, 3)

FeatureGenerator(sliding_window=1)

In [16]:
X_test_stab = feature_generator_stab_test2.transform(X_test_stab, 'stab')
y_test_stab = reduce_label_amount(y_test_stab, 3)

501


In [17]:
clf_stab2 = RandomForestClassifier(n_estimators=150)
clf_stab2.fit(X_train_stab, y_train_stab)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [18]:
pickle.dump(clf_stab2, open("pickled_models/inmotion_randomforest_stab_" + drone_in_use + ".sav", 'wb'))

In [19]:
y_pred_stab = clf_stab2.predict(X_test_stab)

In [20]:
get_confusion_matrix(y_test_stab, y_pred_stab)

,actual_0,actual_1,actual_2
predicted_0,501,0,0
predicted_1,0,346,155
predicted_2,0,76,425


In [21]:
clf_stab2.score(X_test_stab, y_test_stab)

0.846307385229541